# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = pd.read_csv('../output_data/weather_df.csv')

weather_df = file
weather_df

,City,Lat,Lng,Max Temp (°C),Humidity (%),Cloudiness (%),Wind Speed (MPH),Country,Date
0,Kutum,14.2000,24.6667,23.50,66,52,3.96,SD,1631415176
1,Tuktoyaktuk,69.4541,-133.0374,11.00,76,90,3.09,CA,1631415177
2,Saldanha,-33.0117,17.9442,13.03,91,100,4.31,ZA,1631415178
3,Cherskiy,68.7500,161.3000,2.26,50,100,2.49,RU,1631415178
4,Hasaki,35.7333,140.8333,26.16,81,100,1.58,JP,1631414881
...,...,...,...,...,...,...,...,...,...
566,Flagstaff,35.1981,-111.6513,23.83,48,1,1.79,US,1631415333
567,Hua Hin,12.5707,99.9588,26.92,75,100,1.78,TH,1631415624
568,Lusambo,-4.9667,23.4500,23.38,76,100,0.79,CD,1631415624
569,Deputatsky,69.3000,139.9000,11.66,42,68,1.91,RU,1631415625


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]
humidity = weather_df["Humidity (%)"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(0,0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=100,point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Narrow down the cities to fit weather conditions:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.

ideal_weather_df = weather_df.loc[(weather_df["Max Temp (°C)"] < 26.67) & (weather_df["Max Temp (°C)"] > 21.11) & 
                                  (weather_df["Wind Speed (MPH)"] < 10) & (weather_df["Cloudiness (%)"] == 0),:]

# If is null value, drop it
ideal_weather_df = ideal_weather_df.dropna()

# Print out the number of cities and display the dataframe
print(f'There are {len(ideal_weather_df)} ideal cities for vacation')
ideal_weather_df

There are 21 ideal cities for vacation


,City,Lat,Lng,Max Temp (°C),Humidity (%),Cloudiness (%),Wind Speed (MPH),Country,Date
104,Morondava,-20.2833,44.2833,21.23,58,0,5.33,MG,1631415253
109,Alice Springs,-23.7000,133.8833,24.75,14,0,4.63,AU,1631415256
183,Cabra,37.4725,-4.4421,22.17,52,0,0.90,ES,1631415313
207,Carnarvon,-24.8667,113.6333,23.04,28,0,7.72,AU,1631415331
239,Balkanabat,39.5108,54.3671,24.92,16,0,7.22,TM,1631415357
250,San Quintín,30.4833,-115.9500,23.16,81,0,4.01,MX,1631415365
284,Teguise,29.0605,-13.5640,21.12,83,0,6.69,ES,1631415392
289,Zaysan,47.4667,84.8667,22.35,17,0,2.04,KZ,1631415395
314,Changji,44.0167,87.3167,21.44,37,0,1.00,CN,1631415414
325,Qujing,25.4833,103.7833,23.08,52,0,2.46,CN,1631415422


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#Store into variable named hotel_df
hotel_df = ideal_weather_df

# Add Hotel Name column to hotel_df        
hotel_df["Hotel Name"] = ""

#iterate through the data frame,hotel_df
for index, row in hotel_df.iterrows():

    #base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    #Setting parameters with dictionary to search for hotels
    params = {"location": str(row["Lat"]) + "," +str(row["Lng"]),
              "radius": str(5000),
              "type": "Hotel",
              "keyword": "Hotel",
              "key": g_key}
    
    response_json = requests.get(base_url, params=params).json()
  
    try:
        hotel_df.loc[index, "Hotel Name"] = response_json['results'][0]['name']
        

    except (KeyError, IndexError):
        print("No Hotel Found")

No Hotel Found
No Hotel Found


In [7]:
hotel_df

,City,Lat,Lng,Max Temp (°C),Humidity (%),Cloudiness (%),Wind Speed (MPH),Country,Date,Hotel Name
104,Morondava,-20.2833,44.2833,21.23,58,0,5.33,MG,1631415253,Palissandre Cote Ouest resort & SPA
109,Alice Springs,-23.7000,133.8833,24.75,14,0,4.63,AU,1631415256,DoubleTree by Hilton Hotel Alice Springs
183,Cabra,37.4725,-4.4421,22.17,52,0,0.90,ES,1631415313,MS Fuente Las Piedras
207,Carnarvon,-24.8667,113.6333,23.04,28,0,7.72,AU,1631415331,The Gascoyne Hotel
239,Balkanabat,39.5108,54.3671,24.92,16,0,7.22,TM,1631415357,Nebitchi Hotel
250,San Quintín,30.4833,-115.9500,23.16,81,0,4.01,MX,1631415365,Don Eddie's Landing
284,Teguise,29.0605,-13.5640,21.12,83,0,6.69,ES,1631415392,HOTEL BOUTIQUE PALACIO ICO
289,Zaysan,47.4667,84.8667,22.35,17,0,2.04,KZ,1631415395,Zaysan Hotel
314,Changji,44.0167,87.3167,21.44,37,0,1.00,CN,1631415414,Garden Hotel
325,Qujing,25.4833,103.7833,23.08,52,0,2.46,CN,1631415422,Stone Forest International Hotel


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
marker = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))